Machine Learning - Programming Assignment

Health care Multiclass classification Assignment

Student Name: ATHARVA SANJAY DHUMAL

Student ID: 2025ab05230@wilp.bits-pilani.ac.in

Date: 15-02-2026

In [ ]:
# =========================
# SECTION 1: LOAD DATASET FROM GOOGLE DRIVE
# =========================

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Correct dataset path
data_path = "/content/drive/MyDrive/ML Assignment 2/healthcare_dataset.csv"

# Load dataset
df = pd.read_csv(data_path)

print("Dataset Shape:", df.shape)
df.head()


Mounted at /content/drive
Dataset Shape: (1499, 15)


,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,1/31/2024,Matthew Smith,Sons and Miller,Blue Cross,18856.28131,328,Urgent,2/2/2024,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,8/20/2019,Samantha Davies,Kim Inc,Medicare,33643.32729,265,Emergency,8/26/2019,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,9/22/2022,Tiffany Mitchell,Cook PLC,Aetna,27955.09608,205,Emergency,10/7/2022,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,11/18/2020,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.78241,450,Elective,12/18/2020,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,9/19/2022,Kathleen Hanna,White-White,Aetna,14238.31781,458,Urgent,10/9/2022,Penicillin,Abnormal


In [ ]:
# =========================
# SECTION 2: PREPROCESSING
# =========================

from sklearn.preprocessing import LabelEncoder, StandardScaler

# Copy dataset
data = df.copy()

# Encode categorical columns
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# Define features & target
TARGET_COLUMN = data.columns[-1]   # change if your target is different

X = data.drop(columns=[TARGET_COLUMN])
y = data[TARGET_COLUMN]

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

import joblib
joblib.dump(scaler, "scaler.pkl")


print("Target column:", TARGET_COLUMN)

Target column: Test Results


In [ ]:
# =========================
# MULTI-CLASS EVALUATION FUNCTION
# =========================

from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef
)

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)

    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "AUC": roc_auc_score(
            y_test,
            y_prob,
            multi_class='ovr',
            average='weighted'
        ),
        "Precision": precision_score(y_test, y_pred, average='weighted'),
        "Recall": recall_score(y_test, y_pred, average='weighted'),
        "F1 Score": f1_score(y_test, y_pred, average='weighted'),
        "MCC": matthews_corrcoef(y_test, y_pred)
    }

In [ ]:
# =========================
# SECTION 3: TRAIN TEST SPLIT
# =========================

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])

Training samples: 1124
Testing samples: 375


In [ ]:
import pandas as pd
from google.colab import files

# Convert X_test (scaled numpy array) back to DataFrame
X_test_df = pd.DataFrame(X_test, columns=X.columns)

# Attach target values
test_data = X_test_df.copy()
test_data["target"] = y_test.values

# Save as CSV (Excel-readable)
test_data.to_csv("test_samples.csv", index=False)

# Download the file
files.download("test_samples.csv")

print("✅ Testing samples (375 rows) extracted to test_samples.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Testing samples (375 rows) extracted to test_samples.csv


In [ ]:
# =========================
# LOGISTIC REGRESSION
# =========================

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

lr_metrics = evaluate_model(lr, X_test, y_test)
lr_metrics


{'Accuracy': 0.32,
 'AUC': np.float64(0.45857268369706194),
 'Precision': 0.3083227815916603,
 'Recall': 0.32,
 'F1 Score': 0.304650780491442,
 'MCC': np.float64(-0.033103793184101775)}

In [ ]:
# =========================
# DECISION TREE
# =========================

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

dt_metrics = evaluate_model(dt, X_test, y_test)
dt_metrics


{'Accuracy': 0.2906666666666667,
 'AUC': np.float64(0.46603462727426526),
 'Precision': 0.2908538587848933,
 'Recall': 0.2906666666666667,
 'F1 Score': 0.29044607244607246,
 'MCC': np.float64(-0.06735094664794654)}

In [ ]:
# =========================
# K NEAREST NEIGHBORS
# =========================

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

knn_metrics = evaluate_model(knn, X_test, y_test)
knn_metrics


{'Accuracy': 0.29333333333333333,
 'AUC': np.float64(0.4686825223987464),
 'Precision': 0.2853019607843137,
 'Recall': 0.29333333333333333,
 'F1 Score': 0.285535031752465,
 'MCC': np.float64(-0.06784319419392457)}

In [ ]:
# =========================
# NAIVE BAYES
# =========================

from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train, y_train)

nb_metrics = evaluate_model(nb, X_test, y_test)
nb_metrics


{'Accuracy': 0.304,
 'AUC': np.float64(0.4595889820211931),
 'Precision': 0.30007687654148635,
 'Recall': 0.304,
 'F1 Score': 0.2983536775461281,
 'MCC': np.float64(-0.05307991366403321)}

In [ ]:
# =========================
# RANDOM FOREST
# =========================

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)
rf.fit(X_train, y_train)

rf_metrics = evaluate_model(rf, X_test, y_test)
rf_metrics


{'Accuracy': 0.296,
 'AUC': np.float64(0.46116007682987886),
 'Precision': 0.288580989729225,
 'Recall': 0.296,
 'F1 Score': 0.2894283547680592,
 'MCC': np.float64(-0.06419951415443112)}

In [ ]:
# =========================
# XGBOOST
# =========================

!pip install xgboost

from xgboost import XGBClassifier

xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
xgb.fit(X_train, y_train)

xgb_metrics = evaluate_model(xgb, X_test, y_test)
xgb_metrics


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [11:54:35] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


{'Accuracy': 0.3253333333333333,
 'AUC': np.float64(0.4701249956851547),
 'Precision': 0.320732443982444,
 'Recall': 0.3253333333333333,
 'F1 Score': 0.3213233888523915,
 'MCC': np.float64(-0.017641460469888584)}

In [ ]:
# =========================
# FINAL METRICS TABLE
# =========================

results = pd.DataFrame([
    ["Logistic Regression", *lr_metrics.values()],
    ["Decision Tree", *dt_metrics.values()],
    ["KNN", *knn_metrics.values()],
    ["Naive Bayes", *nb_metrics.values()],
    ["Random Forest", *rf_metrics.values()],
    ["XGBoost", *xgb_metrics.values()]
], columns=[
    "Model", "Accuracy", "AUC", "Precision", "Recall", "F1 Score", "MCC"
])

results


,Model,Accuracy,AUC,Precision,Recall,F1 Score,MCC
0,Logistic Regression,0.320000,0.458573,0.308323,0.320000,0.304651,-0.033104
1,Decision Tree,0.290667,0.466035,0.290854,0.290667,0.290446,-0.067351
2,KNN,0.293333,0.468683,0.285302,0.293333,0.285535,-0.067843
3,Naive Bayes,0.304000,0.459589,0.300077,0.304000,0.298354,-0.053080
4,Random Forest,0.296000,0.461160,0.288581,0.296000,0.289428,-0.064200
5,XGBoost,0.325333,0.470125,0.320732,0.325333,0.321323,-0.017641


In [ ]:
import joblib

# Save trained models
joblib.dump(lr, "logistic_regression.pkl")
joblib.dump(dt, "decision_tree.pkl")
joblib.dump(knn, "knn.pkl")
joblib.dump(nb, "naive_bayes.pkl")
joblib.dump(rf, "random_forest.pkl")
joblib.dump(xgb, "xgboost.pkl")

print("All models saved successfully!")


All models saved successfully!


In [ ]:
!ls


decision_tree.pkl  logistic_regression.pkl  sample_data       xgboost.pkl
drive		   naive_bayes.pkl	    scaler.pkl
knn.pkl		   random_forest.pkl	    test_samples.csv


In [ ]:
import joblib
from sklearn.preprocessing import StandardScaler


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=1000)